In [1]:
from openai import OpenAI
import numpy as np

In [ ]:
client = OpenAI(api_key="API_KEY")

In [3]:
with open('nba.txt', 'r') as f:
    text = f.read()
    print(text)

About the NBA
The National Basketball Association, or NBA, is a professional basketball league comprised of 30 teams across North America featuring the best basketball players in the world.

FREQUENTLY ASKED QUESTIONS

How many teams are there in the NBA?

The NBA is a collection of 30 teams, broken down by two conferences with three divisions apiece. The Western Conference plays host to the Northwest, Pacific and Southwest Divisions, featuring the following 15 teams:

Northwest: Denver Nuggets, Minnesota Timberwolves, Oklahoma City Thunder, Portland Trail Blazers, Utah Jazz
Pacific: Golden State Warriors, L.A. Clippers, Los Angeles Lakers, Phoenix Suns, Sacramento Kings
Southwest: Dallas Mavericks, Houston Rockets, Memphis Grizzlies, New Orleans Pelicans, San Antonio Spurs
The Eastern Conferenceâ€™s 15 squads also break up across three divisions: the Atlantic, Southeast and Central:

Atlantic: Boston Celtics, Brooklyn Nets, New York Knicks, Philadelphia 76ers, Toronto Raptors
Central:

In [4]:
CHUNNK_SIZE = 100
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['About the NBA\nThe National Basketball Association, or NBA, is a professional basketball league compr', 'ised of 30 teams across North America featuring the best basketball players in the world.\n\nFREQUENTL', 'Y ASKED QUESTIONS\n\nHow many teams are there in the NBA?\n\nThe NBA is a collection of 30 teams, broken', ' down by two conferences with three divisions apiece. The Western Conference plays host to the North', 'west, Pacific and Southwest Divisions, featuring the following 15 teams:\n\nNorthwest: Denver Nuggets,', ' Minnesota Timberwolves, Oklahoma City Thunder, Portland Trail Blazers, Utah Jazz\nPacific: Golden St', 'ate Warriors, L.A. Clippers, Los Angeles Lakers, Phoenix Suns, Sacramento Kings\nSouthwest: Dallas Ma', 'vericks, Houston Rockets, Memphis Grizzlies, New Orleans Pelicans, San Antonio Spurs\nThe Eastern Con', 'ferenceâ€™s 15 squads also break up across three divisions: the Atlantic, Southeast and Central:\n\nAt', 'lantic: Boston Celtics, Brooklyn Nets, New York 

In [5]:
embedded_chunks = []
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

[('About the NBA\nThe National Basketball Association, or NBA, is a professional basketball league compr', [-0.023602964356541634, -0.0051164994947612286, 0.03132639080286026, 0.035566702485084534, 0.03513401746749878, -0.023408256471157074, -0.013121170923113823, 0.0060521806590259075, -0.029833627864718437, 0.033662889152765274, 0.03904981538653374, -0.007993854582309723, -0.07303722202777863, 0.0059656440280377865, 0.042489662766456604, 0.013791833072900772, -0.060619160532951355, 0.046167485415935516, 0.03372779116034508, -0.01088743470609188, -8.873422302713152e-06, 0.05797978490591049, -0.0328407883644104, -0.0018253896851092577, -0.019286932423710823, -0.013532222248613834, -0.020476816222071648, -0.02420872263610363, -0.001925448072142899, 0.047595344483852386, -0.004878522828221321, -0.02239144593477249, 0.020217204466462135, 0.007409729994833469, 0.06308546662330627, -0.04424203559756279, 0.021623430773615837, -0.003204572945833206, 0.05979705974459648, 0.02059580385684967, 0

In [6]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

In [7]:
question = "why was the nba replay center created?"

In [8]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[-0.017029894515872, -0.01025913842022419, 0.008020531386137009, 0.009229104034602642, -0.06416425108909607, -0.015670249238610268, -0.0014394724275916815, 0.012339807115495205, -0.044579870998859406, -0.013795588165521622, 0.019131164997816086, 0.02018866501748562, -0.03980051353573799, -0.007951862178742886, -0.027426371350884438, -0.02586071938276291, 0.027206631377339363, 0.0012969843810424209, -0.007093500811606646, -0.028676144778728485, -0.0044257123954594135, 0.04372837394475937, -0.01972171664237976, -0.01829340308904648, -0.016906291246414185, 0.006005098111927509, -0.002588818781077862, 0.003931296523660421, -0.002008566167205572, 0.015162100084125996, 0.02919803000986576, -0.021919121965765953, -0.03150530532002449, 0.0054111117497086525, 0.004205971956253052, -0.030598875135183334, -0.0008433402981609106, -0.01672775112092495, 0.029170561581850052, 0.010005063377320766, 0.02507789433002472, 0.011536380276083946, 0.015423041768372059, 0.0035227162297815084, -0.0075123808346

In [9]:
best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

In [ ]:
print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)

Relevant information: with the 2014-15 NBA regular season, the NBA launched its new state-of-the-art NBA Replay Center, ba


In [12]:
print(completion.choices[0].message.content)

The NBA Replay Center was created to improve the accuracy and efficiency of game officiating by providing officials with immediate access to video reviews. Launched with the 2014-15 NBA regular season, the Replay Center allows referees to make more informed decisions by reviewing plays from various camera angles. This technology was introduced to ensure that critical calls, such as those involving last-second shots, boundary violations, and other key moments, are made correctly. The Replay Center aims to enhance the overall fairness and integrity of the game by giving officials the tools they need to review and confirm their on-court decisions effectively.
